In [38]:
import pandas as pd
import numpy as np

import plotly.express as px

### Read-In Data

In [39]:
data_raw = pd.read_csv("../data/census.csv")

In [40]:
data_raw.head(5)

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [41]:
# Remove Space of column name
data_raw.columns = [col.strip() for col in data_raw.columns]

### Check Data Type and Missing Data

In [42]:
df_data_type = pd.DataFrame(data_raw.dtypes)
df_data_type.reset_index(inplace = True)
df_data_type.rename(columns = {'index' : "columns_name", 0: "Data Type"},inplace = True)

df_data_missing = pd.DataFrame(data_raw.isna().mean()*100)
df_data_missing.reset_index(inplace = True)
df_data_missing.rename(columns = {'index' : "columns_name", 0: "Missing Percentage"},inplace = True)

df_data_checking = pd.merge(df_data_type, df_data_missing, on = 'columns_name')

In [43]:
df_data_checking

,columns_name,Data Type,Missing Percentage
0,age,int64,0.0
1,workclass,object,0.0
2,fnlgt,int64,0.0
3,education,object,0.0
4,education-num,int64,0.0
5,marital-status,object,0.0
6,occupation,object,0.0
7,relationship,object,0.0
8,race,object,0.0
9,sex,object,0.0


### Data Check by Columns

#### Categorical Data

In [44]:
categorical_columns = data_raw.select_dtypes("object").columns
numerical_columns = list(set(data_raw.columns).difference(set(categorical_columns)))

In [45]:
def plot_categorical_count(df:pd.DataFrame, column_count:str) -> None:
    count_df = pd.DataFrame(df.groupby([column_count]).size()).reset_index()
    count_df.rename(columns = {0 : 'Records Count'}, inplace = True)
    count_df.sort_values(by = 'Records Count', ascending= False, inplace = True)

    count_plot = px.histogram(count_df, x = column_count , y = "Records Count",
                              color = column_count, width= 700, height=400,
                              title= f"Count of {column_count}")
    count_plot.show()

In [46]:
for col_name in categorical_columns:
    plot_categorical_count(data_raw, col_name)

### Data Cleaning - Categorical Data

In [47]:
# Replace '?' with np.nan to make replace with imputed value
data_raw.replace('?', np.nan, inplace = True)
data_raw.replace('? ', np.nan, inplace = True)
data_raw.replace(' ?', np.nan, inplace = True)

# Replace all NA with mode on each columns
# Remove white space in data
for cat_col in categorical_columns:
    col_mode = data_raw[cat_col].mode()
    data_raw[cat_col] = data_raw[cat_col].str.strip()
    data_raw[cat_col] = data_raw[cat_col].fillna(str(col_mode))

### Categorical Grouping

In [48]:
# Group naitve-country into "United-State" and "Other"
data_raw['native-country'] = np.where(data_raw['native-country'] != 'United-States', 'Other', 'United-States')

# Group race into "White", "Black" and "Other"
data_raw['race'] = np.where((data_raw['race'] == 'Asian-Pac-Islander') |
                            (data_raw['race'] == 'Amer-Indian-Eskimo'), 'Other', data_raw['race'])

data_raw['education'].replace(['11th', '9th', '7th-8th', '5th-6th', 
                               '10th', '1st-4th', 'Preschool', '12th'], 'School', inplace = True)                       

#### Numerical Data

In [49]:
def numerical_hist_plot(data:pd.DataFrame, numerical_col:str) -> None:
    histogram_plot = px.histogram(data, x = numerical_col,
                                  width = 700, height=400,
                                  title= f"Hitogram of {numerical_col}")
    histogram_plot.show()

In [50]:
for col_name in numerical_columns:
    numerical_hist_plot(data_raw, col_name)

In [51]:
data_raw.drop(columns = ['capital-gain', 'capital-loss'], inplace = True)

In [52]:
data_raw.to_csv("../data/census_clean.csv", index = 0)